In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/theunknownsai/stocks/gr500182.csv
/kaggle/input/d/theunknownsai/stocks/gr500209.csv
/kaggle/input/d/theunknownsai/stocks/gr500680.csv
/kaggle/input/d/theunknownsai/stocks/gr530965.csv
/kaggle/input/d/theunknownsai/stocks/gr532174.csv
/kaggle/input/d/theunknownsai/stocks/gr532210.csv
/kaggle/input/d/theunknownsai/stocks/gr532540.csv
/kaggle/input/d/theunknownsai/stocks/gr500325.csv
/kaggle/input/d/theunknownsai/stocks/gr500112.csv
/kaggle/input/d/theunknownsai/stocks/gr500180.csv
/kaggle/input/d/theunknownsai/stocks/gr507685.csv


In [3]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.backend import sigmoid
import matplotlib.pyplot as plt
from math import sqrt
import threading
from multiprocessing.pool import ThreadPool
import time
import multiprocessing
import traceback

import prettytable

In [4]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [ ]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [ ]:
def reshape_data(x_train, x_test, y_train, y_test):
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0],1, x_train.shape[1]))
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0],1, x_test.shape[1]))
    y_test = np.array(y_test)
    y_test = np.reshape(y_test, (y_test.shape[0],1))

    return (x_train, x_test, y_train, y_test)

In [ ]:
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

In [ ]:
combinations = list()
def get_combinations(arr):
    n = len(arr)
    indices = [0 for i in range(n)]
    while (1):
        res = list()
        for i in range(n):
            res.append(arr[i][indices[i]])
        combinations.append(res)
        next = n - 1
        while (next >= 0 and
              (indices[next] + 1 >= len(arr[next]))):
            next-=1
        if (next < 0):
            return
        indices[next] += 1
        for i in range(next + 1, n):
            indices[i] = 0
u = [25,75,50,100]
b = [25,32,48,64]
e = [25,50,75,100]
arr = [u,b,e]
get_combinations(arr)

In [ ]:
columns = ["units","epochs","batch_size","layers","direction","root_mean_squared_error","mean_absolute_error","mean_squared_error","rsquared_adj"]
tables = {}
layers = 3
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        tables[int(filename[2:8])] = prettytable.PrettyTable()
        tables[int(filename[2:8])].field_names = columns
        tables[int(filename[2:8])].title = str(filename[2:8]) + " " + "RNN"

In [ ]:
def recurrent_neural_networks(X,Y,layers,epochs = 5,batch_size = 32,units=50,filename=filename):
    cur = (units,batch_size,epochs)
    print(cur)
        
    
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
    x_train, x_test, y_train, y_test = reshape_data(x_train, x_test, y_train, y_test)

    model = build_lstm(x_train,layers,units)
    model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
    
    history_lstm = model.fit(x_train,y_train,epochs = epochs,batch_size = batch_size, validation_data = (x_test,y_test),shuffle = False,verbose=0)

    y_pred = model.predict(x_test)
    y_pred = np.array(y_pred)
    y_pred = np.reshape(y_pred, (y_pred.shape[0],1))

    rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)

    c = 0
    for a,b in zip(y_pred,y_test):
        if a * b >= 0:
            c += 1
    direction = c/len(y_test)
    try:
        tables[filename].add_row([units,epochs,batch_size,layers,direction,rmse,mae,mse,r2])
    except :
        pass
    myres =  {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse,"rsquared_adj":r2}
    myres.update({"units":units,"epochs":epochs,"batch_size":batch_size,"layers":layers,"direction":direction})
    print("done")
    return {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse,"rsquared_adj":r2,"same direction":c,"total":len(y_test)}


In [ ]:
def build_lstm(X_train,layers,units = 50):
    model = Sequential()
    model.add(LSTM(units=units,return_sequences=True,input_shape=(1,X_train.shape[2])))
    model.add(Dropout(0.2))
    for i in range(layers):
        model.add(LSTM(units=units,return_sequences=True))
        model.add(Dropout(0.2))
    model.add(Dense(units = 1,activation = 'swish'))
    return model

In [ ]:
%%time
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        filepath = os.path.join(dirname, filename)
        if filename.startswith("gr"):
            print(filename)
            try:
                df = pd.read_csv(filepath)
                column = "Next Day Close Price GR"
                df = pre_process_data(df,60)
                (data,column) = dependent_column(df,column)
                X = data[data.columns[:-1]]
                Y = data[column]
                arguments = list()
                for units,batch_size,epochs in combinations:
#                     data = [X,Y,layers,epochs,batch_size,units,int(filename[2:8])]
                    result = recurrent_neural_networks(X,Y,layers,epochs,batch_size,units,int(filename[2:8]))
#                     arguments.append(data)
#                 threads = ThreadPool(4)
#                 result = threads.starmap(recurrent_neural_networks,arguments)
                s = tables[int(filename[2:8])].get_string()
                result = [tuple(filter(None, map(str.strip, splitline))) for line in s.splitlines() for splitline in [line.split("|")] if len(splitline) > 1]
                resultdf = pd.DataFrame(result[2:],columns=result[1])
                resultdf.to_csv(os.path.join(os.getcwd(),str(filename[2:8])+"_rnn"+".csv"),index=None)
            except Exception as e :
                traceback.print_exc()
                print(e)
                pass

In [8]:
path = os.path.join(os.getcwd(),"Results")

In [30]:
best_rnn = pd.DataFrame()
for dirname, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith("_rnn.csv"):
            filepath = os.path.join(dirname, filename)
            df = pd.read_csv(filepath)
            # df.drop(columns=["layers"],inplace=True)
            table = prettytable.PrettyTable()
            table.field_names = df.columns
            table.title = str(filename[:6]) + " rnn"
            for _,row in df.iterrows():
                row = [round(r,6) for r in row]
                table.add_row(row)
            print(table)
            df = df.sort_values(by=["direction"],ascending=[False])
            df["security id"] = filename[:6]
            best_rnn = best_rnn.append(df.head(1),ignore_index=True)
best_rnn.to_csv(os.path.join(path,"best_rnn"+".csv"),index=None)

+-----------------------------------------------------------------------------------------------------------------------------+
|                                                          500112 rnn                                                         |
+-------+--------+------------+-----------+-------------------------+---------------------+--------------------+--------------+
| units | epochs | batch_size | direction | root_mean_squared_error | mean_absolute_error | mean_squared_error | rsquared_adj |
+-------+--------+------------+-----------+-------------------------+---------------------+--------------------+--------------+
|  25.0 |  25.0  |    25.0    |  0.481069 |         0.293117        |       0.028221      |      0.085918      |  -0.003941   |
|  25.0 |  50.0  |    25.0    |  0.478842 |         0.292802        |       0.027615      |      0.085733      |  -0.001781   |
|  25.0 |  75.0  |    25.0    |  0.506682 |         0.29271         |       0.02778       |      0.08567

KeyError: "['layers'] not found in axis"

In [42]:
besttable = prettytable.PrettyTable()
besttable.title = "best rnn"
besttable.field_names = ["security id","units","epochs","batch_size","direction"]
for _,row in best_rnn.iterrows():
    besttable.add_row(row[besttable.field_names])
print(besttable)

+----------------------------------------------------------------+
|                            best rnn                            |
+-------------+-------+--------+------------+--------------------+
| security id | units | epochs | batch_size |     direction      |
+-------------+-------+--------+------------+--------------------+
|    500112   |   25  |  100   |     25     | 0.5133630289532294 |
|    500180   |  100  |   50   |     48     | 0.5295429208472687 |
|    500182   |   25  |  100   |     25     | 0.5177777777777778 |
|    500209   |  100  |  100   |     25     | 0.5350389321468298 |
|    500325   |   75  |  100   |     64     | 0.5094549499443827 |
|    500680   |  100  |  100   |     64     |       0.5125       |
|    507685   |   50  |  100   |     32     | 0.5502232142857143 |
|    530965   |  100  |   25   |     25     | 0.5441340782122905 |
|    532174   |  100  |   75   |     25     | 0.5155902004454342 |
|    532210   |   50  |   75   |     64     | 0.55184331797235